Content-based Recommendation System
- Simple genre-genre recommendation
- First, the user does Spotify authorization 
- Upon authorization they can make calls to the Spotify API
- They will: 1. send over a list of their top genres, 2. send over a list of their top artists.
- In the database, there is a list of all artists, the user's top artists will be concatenated with that list
- Then, all-pairs matrix is constructed from that entire list

- Assumptions: the database has diff artists than the user's current spotify listings, also that whatever recs we return can be in the database or in the database and on spotify but not solely on spotify
- These assumptions are faulty. For the former, I'd have to do a join or something to get realistic results, and the latter should hold but kind of contingent on db being big enough?

In [102]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [126]:
user_genres = ['rock', 'pop', 'rnb', 'rap'] # this doesnt rly make sense if i'm collating based on fav artists just fyi
user_artists = ['lil tay', 'taylor swift']

# insert logic to fetch user_artist's genres 

user_stuff = pd.DataFrame({
    "username": pd.Series(["lil tay", "taylor swift"]), 
    "genre": pd.Series([['rap'], ['pop']])
})

artist_genres = pd.DataFrame({
    "username": pd.Series(["rich brian", "chief keef", "swae lee"]),
    "genre": pd.Series([['rock', 'rap', 'trap'], ['blues', 'shoegaze', 'rap'], ["pop", "trap", "classical"]]) 
})

artist_genres = pd.concat([user_stuff, artist_genres]).reset_index(drop=True)

reverse_artist_to_idx = {}
for i in range(len(artist_genres)): 
    reverse_artist_to_idx[artist_genres.loc[i, "username"]] = i

print(artist_genres)
print(reverse_artist_to_idx)

       username                   genre
0       lil tay                   [rap]
1  taylor swift                   [pop]
2    rich brian       [rock, rap, trap]
3    chief keef  [blues, shoegaze, rap]
4      swae lee  [pop, trap, classical]
{'lil tay': 0, 'taylor swift': 1, 'rich brian': 2, 'chief keef': 3, 'swae lee': 4}


In [127]:
c = list(set(np.append(np.array(user_genres), np.concatenate(artist_genres["genre"])))) # genre vocabulary

reverse_lookup = {}
for i in range(len(c)):
    reverse_lookup[c[i]]=i 

In [137]:
def cosine_distance(arr1, arr2): 
    return 1 - (np.dot(arr1, arr2) / (np.sqrt(np.sum(arr1)) + np.sqrt(np.sum(arr2))))

ohe_artist_genres = {"username": [], "genres": []}
for i, genres in enumerate(list(artist_genres["genre"])): 
    username = artist_genres.loc[i, "username"]
    ohe_vec = np.zeros((len(c)))
    for genre in genres:
        ohe_vec[reverse_lookup[genre]]=1 
    ohe_artist_genres["username"].append(username)
    ohe_artist_genres["genres"].append(ohe_vec)
ohe_artist_genres = pd.DataFrame(ohe_artist_genres)

# allpairs matrix--assumes everything is in order of the original DF's username entries for back-lookups
m = len(ohe_artist_genres["genres"])
all_pairs=np.zeros((m, m))
for i in range(len(ohe_artist_genres["genres"])): 
    for j in range(len(ohe_artist_genres["genres"])):
        all_pairs[i][j] = cosine_distance(ohe_artist_genres["genres"][i], ohe_artist_genres["genres"][j])
        
print(all_pairs)

[[0.5        1.         0.6339746  0.6339746  1.        ]
 [1.         0.5        1.         1.         0.6339746 ]
 [0.6339746  1.         0.1339746  0.71132487 0.71132487]
 [0.6339746  1.         0.71132487 0.1339746  1.        ]
 [1.         0.6339746  0.71132487 1.         0.1339746 ]]


In [148]:
model_knn= NearestNeighbors(metric='precomputed', algorithm='brute', n_neighbors=3, n_jobs=-1)

In [149]:
model_knn.fit(all_pairs)

NearestNeighbors(algorithm='brute', metric='precomputed', n_jobs=-1,
                 n_neighbors=3)

In [150]:
# insert logic saying -- IF only in database THEN consider the rankings
# print(all_pairs[0].reshape(1,-1).shape)
distances, indices = model_knn.kneighbors(all_pairs[2].reshape(1,-1), n_neighbors=3)

In [151]:
print(distances, indices) # i need to resolve recommending itself, decrease similarity of input? dk, can just manually get rid of this

[[0.1339746  0.6339746  0.71132487]] [[2 0 3]]


In [158]:
sorted_vals = sorted(zip(distances, indices), key=lambda x:x[0]) # sort by distances in increasing order (most similar first)

In [160]:
print("Recommendations: ", sorted_vals[0][1])

Recommendations:  [2 0 3]
